# BLNK Analysis Pipeline
This notebook serves an interactive means to operate the BLNK extraction pipeline. Following the steps below, 
one may analyze videos of the eye and extract pupil and eyelids information therefrom, outputting the 
result as a MATLAB file. 

## Step 1: Environment Setup and Library Imports
To use the pipeline, one must first install [Pylids](https://github.com/piecesofmindlab/pylids), which is attached as a submodule to this GitHub repo. 
Follow their installation instructions first. This will generate a pylids conda environment you will use as the basis for this project into which
we will install further libraries.  

### Ensuring Correct Kernel
To ensure you are using the correct pylids kernel, we will first import some standard library functions to check 
the name of the kernel you are using and assert it is the correct one

In [1]:
import os 
import sys
import pathlib
import importlib
conda_kernel_name: str | None = os.environ.get("CONDA_DEFAULT_ENV")
assert conda_kernel_name == "pylids", f"Conda environment: {conda_kernel_name} is not equal to pylids"

### Installing Additional Libraries. 
After the pylids conda environment has been created, we have to also install 
additional libraries into that environment used by this project 

In [ ]:
%pip install pupil-detectors
%pip install pye3d
%pip install natsort 
%pip install hdf5storage
%pip install scipy 
%pip install dill
%pip install h5py

### Import Custom Libraries
Next, we will import the custom libraries written to do the analysis for this pipeline

In [2]:
# Define the path to the top level of htis repo 
blnk_analysis_dir: str = "/Users/zacharykelly/Documents/MATLAB/projects/BLNK_pipeline"
assert os.path.exists(blnk_analysis_dir), f"Path: {blnk_analysis_dir} does not exist"
sys.path.append(blnk_analysis_dir)

from utility import video_io
from blnk_analysis_code.utility import blnk_analysis_pipeline

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Step 2: Gather Paths to Videos for Analysis
Now that our libraries are properly installed, we will not gather the path(s) to the videos that 
we would like to analyze

In [3]:
# First, we will initialize the list of videos we want to analyze. 
# We will populate this list with either a single path or many paths later 
videos_to_analyze: list[str] = []

In [ ]:
# If you simly want to analyze a single video, enter the path to the 
# video directory here 
single_video_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_raw/PuffLight/modulate/TEST_001/TEST_001_modulate_direction-Mel_contrast-0.40_phase-3.14_trial-001_dual.avi"
videos_to_analyze.append(single_video_path)

In [4]:
# If you want to analyze a directory of videos, simply enter the path to that directory here 
video_directory_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_raw/PuffLight/modulate/BLNK_1007"
videos_to_analyze.extend([os.path.join(video_directory_path, filename) 
                          for filename in os.listdir(video_directory_path) if filename.endswith(".avi")]
                        )

In [6]:
# Next, we will assert that all paths are unique, exist and can be accessed (e.g. ensure they are local files if the directory is on DropBox)
unique_videos_to_analyze: dict[str, None] = {video: None 
                                             for video in videos_to_analyze
                                            } # We use a dictionary to preserve order as compared to set

# After we have asserted that all of the videos are unique, let's ensure the paths exist and they can be accessed 
for path in unique_videos_to_analyze:
    assert os.path.exists(path), f"Path does not exist: {path}"
    try:
        assert video_io.inspect_video_frame_count(path) != 0, f"Frame count for path: {path} is 0" 
    except:
        raise Exception(f"Cannot count frames for path: {path}. Check if the file is corrupted/online-only?")
    
# Reassign videos to analyze to be the unique set 
videos_to_analyze = [video for video, _ in unique_videos_to_analyze.items()]
assert len(videos_to_analyze) != 0, f"No videos to analyze"

## Step 3: Defining Output Location
Next, we will define where the output of the analysis of the desired videos should be placed. 

In [ ]:
# Generate output folder 
output_folder_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_analysis/PuffLight/modulate/BLNK_1007"
os.makedirs(output_folder_path, exist_ok=True)
assert os.path.exists(output_folder_path)

## Step 4: Analyze Videos 
Now, we will analyze the desired videos. Doing so involves an intermediate step of modifying the video 
video cropping and then padding to a certain size, while also blacking out overly white pixels. Before analyzing, we must define the values of these parameters.

### Parameter Declaration

In [8]:
# Define the crop box and target size 
t, b, l, r = 0, 240, 0, 325
crop_box: tuple = (t, b, l, r)
target_size: tuple = (480, 640)
whiteness_threshold: int = 225

# Generate a temp output directory for the intermediate stage videos
temp_dir_path: str = './temp_blnk_pipeline'
if(not os.path.exists(temp_dir_path)):
    os.mkdir(temp_dir_path)


### Analysis

In [10]:
importlib.reload(blnk_analysis_pipeline)

# Iterate over the videos and output them to the target directory 
for video_num, video_path in enumerate(videos_to_analyze):
    print(f"Processing video: {video_num+1}/{len(videos_to_analyze)}", flush=True)
    blnk_analysis_pipeline.predict_eye_features(video_path, output_folder_path, crop_box, target_size, temp_dir_path, 
                                                threshold_value=whiteness_threshold,
                                                visualize_results=False
                                               )
    

Processing video: 1/34
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_BLNK_1007_modulate_adapt-05_L.avi
Video metadata: 
  Overall # of frames:    9900
  Duration of video [s]:  55.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 9900/9900 [05:31<00:00, 29.88it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp958xvy2n/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp958xvy2n/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp958xvy2n/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-eyelid-v1/dlc-models-pytorch/iteration-0/sant

100%|██████████| 9900/9900 [07:18<00:00, 22.60it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp5iv3g3fb/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp5iv3g3fb/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp5iv3g3fb/temp_BLNK_1007_modulate_adapt-05_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5']
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santi

...

Processing video: 2/34
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_BLNK_1007_modulate_direction-LMS_contrast-0.20_phase-0.00_trial-001_L.avi
Video metadata: 
  Overall # of frames:    11160
  Duration of video [s]:  62.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 11160/11160 [06:11<00:00, 30.07it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpm9t6d7pt/temp_BLNK_1007_modulate_direction-LMS_contrast-0.20_phase-0.00_trial-001_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpm9t6d7pt/temp_BLNK_1007_modulate_direction-LMS_contrast-0.20_phase-0.00_trial-001_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpm9t6d7pt/temp_BLNK_1007_modulate_direction-LMS_contrast-0.20_phase-0.00_trial-001_LDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
Model weights already exist!
Analyzing 

 88%|████████▊ | 9831/11160 [07:09<00:58, 22.86it/s]


KeyboardInterrupt: 